In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import time
import DenseModule

In [2]:
#import, load and normalize CIFAR

trainBatchSize = 128
testBatchSize = 128

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=trainBatchSize,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=testBatchSize,
                                         shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat',
          'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
#constants and net definition
learning_rate = 0.002
numEpochs = 300
lr_decay = 0.995
net = DenseModule.DenseLinearlyInvariantNet(32, 4, 10, 4, 4)

In [4]:
def testPerformance(dataLoader, dataName):
    correct = 0
    total = 0
    loss = 0
    testCriterion = nn.CrossEntropyLoss(size_average=False)
    with torch.no_grad():
        net.eval()
        for data in dataLoader:
            inputs, labels = data
            if cuda_avail:
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            outputs = net(inputs)
            loss += testCriterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Loss and accuracy on the %s set: %.5f, %.2f %%' % (dataName, loss / total, 100 * correct / total))

In [ ]:
#Train on training set
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

def adjust_learning_rate():
    global learning_rate
    learning_rate *= lr_decay
    for param_group in optimizer.param_groups:
        param_group["lr"] = learning_rate

cuda_avail = torch.cuda.is_available()
if cuda_avail:
    net.cuda()

net.printNet()
print("Number of epochs: ", numEpochs)
for epoch in range(numEpochs):  # loop over the dataset multiple times
    timeStart = time.time()
    net.train()
    running_loss = 0.0
    numBatches = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        
        if cuda_avail:
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
            
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        numBatches += 1
    timeElapsed = time.time() - timeStart
    adjust_learning_rate()
        
    print('[%d] loss: %.3f LR: %.5f Epoch time: %.2f s, Remaining time: %.2f s' %
          (epoch + 1, running_loss / numBatches, learning_rate, timeElapsed, (numEpochs - epoch - 1) * timeElapsed))
    
    if ((epoch + 1) % 10 == 0):
        testPerformance(trainloader, "train")
        testPerformance(testloader, "test")

print('Finished Training')
testPerformance(trainloader, "train")
testPerformance(testloader, "test")

Dense Linearly Invariant Net will be trained!
Number of parameters:  43674
Number of epochs:  300
[1] loss: 2.165 LR: 0.00199 Epoch time: 6.32 s, Remaining time: 1890.91 s
[2] loss: 1.937 LR: 0.00198 Epoch time: 6.33 s, Remaining time: 1886.08 s
[3] loss: 1.776 LR: 0.00197 Epoch time: 6.35 s, Remaining time: 1885.89 s
[4] loss: 1.682 LR: 0.00196 Epoch time: 6.40 s, Remaining time: 1894.09 s
[5] loss: 1.606 LR: 0.00195 Epoch time: 6.39 s, Remaining time: 1886.28 s
[6] loss: 1.552 LR: 0.00194 Epoch time: 6.41 s, Remaining time: 1884.74 s
[7] loss: 1.503 LR: 0.00193 Epoch time: 6.44 s, Remaining time: 1887.78 s
[8] loss: 1.467 LR: 0.00192 Epoch time: 6.39 s, Remaining time: 1867.32 s
[9] loss: 1.425 LR: 0.00191 Epoch time: 6.41 s, Remaining time: 1865.99 s
[10] loss: 1.390 LR: 0.00190 Epoch time: 6.45 s, Remaining time: 1869.88 s
Loss and accuracy on the train set: 1.35074, 53.43 %
Loss and accuracy on the test set: 1.45726, 49.84 %
[11] loss: 1.359 LR: 0.00189 Epoch time: 6.41 s, Remaini

In [ ]:
#Distribution over classes in test set

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    net.eval()
    for data in testloader:
        inputs, labels = data
        if cuda_avail:
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %.2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))